<a href="https://colab.research.google.com/github/Onmang/Multimedia_engineering/blob/master/T20ME022_ex07_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/resistor_v3.zip"

In [2]:
# ライブラリのインポート
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os # ディレクトリ（フォルダ）やファイルを扱うためのライブラリ（本当はPathlibライブラリのほうが良いが難しいので簡単な方で）
import glob # ファイル一覧を取得するためのライブラリ
import re # 正規表現を使ったパターンマッチング用（ラベルを取得するため）
tf.test.gpu_device_name() # GPUの利用確認

'/device:GPU:0'

In [15]:
train_list = glob.glob('data/train/*/*.png') # 訓練用画像ファイルの取得（拡張子がpng）
valid_list = glob.glob('data/valid/*/*.png') # 検証用画像ファイルの取得
classes = sorted(os.listdir('data/train'), key=int) # 教師ラベルの一覧をリストで取得する。"sorted"でソートしておく。
print(classes) # 取得した抵抗器のラベルを表示。文字列になっている点に注意すること！

['150', '160', '390', '430', '620', '1600', '1800', '2200', '2400', '3000', '3300', '3600', '5600', '9100']


In [19]:
train_ds = tf.data.Dataset.list_files(train_list) # 訓練ファイル名のリストをTensor型に変換
valid_ds = tf.data.Dataset.list_files(valid_list) # テストファイル名のリストをTensor型に変換

In [20]:
train_ds = tf.data.Dataset.list_files(train_list) # 訓練ファイル名のリストをTensor型に変換
valid_ds = tf.data.Dataset.list_files(valid_list) # 検証用のファイル名のリストをTensor型に変換

# ファイル名から画像データをロードしてNNへ入力できるようにデータを成形する。ついでに教師ラベルも取得する
def load_file(files):
    ys = [] # ラベル
    xs = [] # 入力データ
    for f in files:
        file = bytes.decode(f.numpy()) # ファイル名はTensor型で保存されているため，文字列型として取得する。
        m = re.search(r'/(\d+)_', file) # 正規表現を使ってファイル名から抵抗値を取得する。
        label = m.groups()[0] # 抵抗値を取得しlabelに保存
        ys.append(label) # ラベルをラベルリストに追加する
        img = cv2.imread(file) # 画像データをカラーで取得。画像サイズは64x64になっているのでここではリサイズしない。
        xs.append(img) # データを入力データリストに追加
    xs = np.array(xs, dtype=np.float32) / 255. # 正規化してfloat32の行列に変換する
    ys = np.array(ys, dtype=np.float32) # ラベルも行列に変換
    return xs, ys

#
# tf.Dataによるtf.Tensor変換
#
AUTOTUNE = tf.data.experimental.AUTOTUNE # 処理を最適化するためのおまじない（自動チューニング設定）
train_ds = train_ds.shuffle(len(train_list)) # 訓練データをシャッフルする。引数にはデータ数を指定すると完全なシャッフルが行われる。len(x_train)は60000。
train_ds = train_ds.repeat(1) # 1 epochで使われるデータの回数。1の場合，1epochで1回しか使われない。引数を空欄にすると無限に使われる。
train_ds = train_ds.batch(32) # ミニバッチを作る。1バッチ32個のデータ。
train_ds = train_ds.map(lambda files: tf.py_function(load_file, [files], Tout=[tf.float32, tf.float32])) # ファイル名から入力ラベルとラベルを取得
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE) # 訓練中に次のバッチを取り出すための処理。

valid_ds = valid_ds.batch(32) # 検証データはシャッフルする必要ないので，バッチ化のみの処理でOK。
valid_ds = valid_ds.map(lambda x: tf.py_function(load_file, [x], Tout=[tf.float32, tf.float32]))

In [21]:
# Functional API
input = Input(shape=(64, 64, 3), name='input') # 入力層の定義　64×64×3 （カラー画像）
h = Conv2D(16, (3, 3), padding='same', activation='relu', name='cnn01')(input)
h = MaxPooling2D((2, 2), name='pool01')(h)
h = Conv2D(32, (3, 3), padding='same', activation='relu', name='cnn02')(h)
h = MaxPooling2D((2, 2), name='pool02')(h)
h = Conv2D(64, (3, 3), padding='same', activation='relu', name='cnn03')(h)
h = MaxPooling2D((2, 2), name='pool03')(h)
h = Conv2D(128, (3, 3), padding='same', activation='relu', name='cnn04')(h)
h = MaxPooling2D((2, 2), name='pool04')(h)
h = Conv2D(256, (3, 3), padding='valid', activation='relu', name='cnn05')(h)
h = MaxPooling2D((2, 2), name='pool05')(h)
h = Flatten(name='flatten')(h) # GlobalAveragePoolingでも良い
h = Dense(128, activation='relu', name='dense01')(h) # 全結合層の隠れ層のノードは128
output = Dense(1, activation='linear', name='output')(h) # 出力層

model = Model(inputs=input, outputs=output) # この処理でモデルを実体化する。入力層と出力層を渡すと自動的にネットワークを構築してくれる。
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 cnn01 (Conv2D)              (None, 64, 64, 16)        448       
                                                                 
 pool01 (MaxPooling2D)       (None, 32, 32, 16)        0         
                                                                 
 cnn02 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 pool02 (MaxPooling2D)       (None, 16, 16, 32)        0         
                                                                 
 cnn03 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 pool03 (MaxPooling2D)       (None, 8, 8, 64)          0   

In [ ]:
# TFのバグでこのように書く
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練の実施
model.fit(train_ds, epochs=500, validation_data=valid_ds)

In [31]:
# 評価
errors = []
for file in glob.glob('data/test/*.png'):
    m = re.search(r'/(\d+)_', file) # ちょっと違うパターンの書き方
    label = float( m.groups()[0] ) # 実数に変換
    img = cv2.imread(file)
    img = img.reshape(1, 64, 64, 3)
    img = np.float32(img) / 255.
    pred = model.predict(img)
    estimate = pred[0][0]
    error = abs(label - estimate) / label * 100
    print(f'Label {label}, Estimate {estimate}, Error Rate: {error}')
    errors.append(error)
ave = np.average(np.array(errors))
print(f'平均誤り率 {ave:.2f}%')

1/1 [==============================] - 0s 88ms/step
Label 2400.0, Estimate 2396.937255859375, Error Rate: 0.12761433919270834
1/1 [==============================] - 0s 20ms/step
Label 3000.0, Estimate 3014.156982421875, Error Rate: 0.4718994140625
1/1 [==============================] - 0s 18ms/step
Label 620.0, Estimate 637.8132934570312, Error Rate: 2.873111847908266
1/1 [==============================] - 0s 18ms/step
Label 160.0, Estimate 166.12399291992188, Error Rate: 3.8274955749511714
1/1 [==============================] - 0s 22ms/step
Label 1800.0, Estimate 1820.3985595703125, Error Rate: 1.1332533094618056
1/1 [==============================] - 0s 21ms/step
Label 1600.0, Estimate 1605.006103515625, Error Rate: 0.3128814697265625
1/1 [==============================] - 0s 17ms/step
Label 430.0, Estimate 439.8010559082031, Error Rate: 2.279315327489099
1/1 [==============================] - 0s 19ms/step
Label 5600.0, Estimate 5652.86962890625, Error Rate: 0.9441005161830357
1/1 [=